# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


C:\Users\Prolex\Anaconda3\envs\Python27\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [24]:
# frequency table for prestige and whether or not someone was admitted
print "Prestige:"
print df.prestige.value_counts()

Prestige:
2.0    148
3.0    121
4.0     67
1.0     61
Name: prestige, dtype: int64


In [23]:
print "Admit:"
print df.admit.value_counts()

Admit:
0    271
1    126
Name: admit, dtype: int64


In [6]:
df.describe()

,admit,gre,gpa,prestige
count,397.000000,397.000000,397.000000,397.000000
mean,0.317380,587.858942,3.392242,2.488665
std,0.466044,115.717787,0.380208,0.947083
min,0.000000,220.000000,2.260000,1.000000
25%,0.000000,520.000000,3.130000,2.000000
50%,0.000000,580.000000,3.400000,2.000000
75%,1.000000,660.000000,3.670000,3.000000
max,1.000000,800.000000,4.000000,4.000000


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [19]:
dummies = pd.get_dummies(df.prestige, prefix = "prestige")

df_1 = df.loc[:,'admit':'gpa']
df_dummies = df_1.join(dummies)

print df_dummies


     admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  \
0        0  380.0  3.61             0             0             1   
1        1  660.0  3.67             0             0             1   
2        1  800.0  4.00             1             0             0   
3        1  640.0  3.19             0             0             0   
4        0  520.0  2.93             0             0             0   
5        1  760.0  3.00             0             1             0   
6        1  560.0  2.98             1             0             0   
7        0  400.0  3.08             0             1             0   
8        1  540.0  3.39             0             0             1   
9        0  700.0  3.92             0             1             0   
10       0  800.0  4.00             0             0             0   
11       0  440.0  3.22             1             0             0   
12       1  760.0  4.00             1             0             0   
13       0  700.0  3.08           

#### 2.2 When modeling our class variables, how many do we need? 



Answer: 

Only 3 dummies variables are required for modeling, because if all three variables are false or 0, the fourth variable is definitely true.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [36]:
cols_to_keep = ['admit', 'gre', 'gpa']

#define dummies variables
dummy_ranks = pd.get_dummies(df.prestige, prefix = "prestige")
handCalc = df[cols_to_keep].join(dummy_ranks.loc[:, 'prestige_1':])
handCalc.head()

,admit,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1


In [39]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(handCalc.admit, handCalc['prestige_1.0'])

prestige_1.0,0,1
admit,,
0,243,28
1,93,33


In [41]:
#Crosstab for all prestige vs admit
pd.crosstab(handCalc.admit, [handCalc['prestige_1.0'],handCalc['prestige_2.0'],handCalc['prestige_3.0'],handCalc['prestige_4.0']])

prestige_1.0   0           1
prestige_2.0   0       1   0
prestige_3.0   0   1   0   0
prestige_4.0   1   0   0   0
admit                       
0             55  93  95  28
1             12  28  53  33

#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [47]:
# Total event -> 93 + 33
# Odd = Y/(1-Y) ; Y is probability
# Odd = instances of event / (total number of instance - instances of event)

odds_1 = 33/float(397 - 33)
print odds_1

0.0906593406593


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [54]:
odds_2 = 93/float(397 - 93)
print odds_2

0.305921052632


#### 3.3 Calculate the odds ratio

In [49]:
# odd ratio = odd_1 / odd_2

odd_ratio1 = odds_1/odds_2
print odd_ratio1

0.296348812478


#### 3.4 Write this finding in a sentenance: 

Answer: 



#### 3.5 Print the cross tab for prestige_4

In [50]:
pd.crosstab(handCalc.admit, handCalc['prestige_4.0'])

prestige_4.0,0,1
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [51]:
odd_ratio2 = (12/float(397-12))/(114/float(397-114))
print odd_ratio2

0.0773752563226


#### 3.7 Write this finding in a sentence

Answer:

## Part 4. Analysis

In [ ]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.loc[:, 'prestige_2':])
print data.head()

We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [ ]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [ ]:
train_cols = []

#### 4.2 Fit the model

#### 4.3 Print the summary results

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

#### 4.5 Interpret the OR of Prestige_2

Answer: 

#### 4.6 Interpret the OR of GPA

Answer: 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [ ]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [ ]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

#### 5.1 Recreate the dummy variables

In [ ]:
# recreate the dummy variables

# keep only what we need for making predictions


#### 5.2 Make predictions on the enumerated dataset

#### 5.3 Interpret findings for the last 4 observations

Answer: 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.